In [1]:
!pip install nltk
!pip install textblob
!pip install textblob syllables
!pip install pyphen

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.3/939.3 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 6.6.0
    Uninstalling importlib-metadata-6.6.0:
      Successfully uninstalled importlib-metadata-6.6.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.8 MB/s eta 0:00:00


In [8]:
from textblob import TextBlob
import syllables
from syllables import estimate


In [9]:
import re
import nltk
from nltk.corpus import stopwords
from textblob import TextBlob
from syllables import estimate 
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
import pyphen

dic = pyphen.Pyphen(lang='en')

def syllables(word):
    """
    Count the number of syllables in a word using Pyphen.
    """
    return len(dic.inserted(word).split('-'))

In [11]:
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup

from nltk.corpus import stopwords
from textblob import TextBlob, Word

# download the stop words list
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# read the Excel file containing URLs
df = pd.read_excel("Input.xlsx")

# create lists to store the results
titles = []
texts = []
positive_scores = []
negative_scores = []
polarity_scores = []
subjectivity_scores = []
fog_indexes = []
average_sentence_lengths = []
percentage_complex_word = []
average_word_lengths = []
syllable_counts = []
personal_pronouns_counts = []
word_counts = []


# iterate over each row in the DataFrame
for index, row in df.iterrows():
    url = row["URL"]
    
    # fetch the webpage content using urllib
    try:
        html = urllib.request.urlopen(url).read()
    except:
        print(f"Invalid URL: {url}")
        continue

    # parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html, "html.parser")

    # extract the article title
    title = soup.title.string
    titles.append(title)

    # extract the article text
    text = ""
    for article in soup.find_all("article"):
        for p in article.find_all("p"):
            text += p.get_text()
    texts.append(text)

#POSITIVE

    positive_words = "/content/positive-words.txt"
    with open(positive_words, "r") as f:
        positive_words = f.read().splitlines()

    negative_words = "/content/negative-words.txt"
    
    with open('/content/negative-words.txt', 'r', encoding='utf-8', errors='ignore') as f:
        negative_words = set(f.read().split())
    


    # calculate positive and negative scores
    positive_score = 0
    negative_score = 0
    words = text.split()
    for word in words:
        if word in positive_words:
            positive_score += 1
        elif word in negative_words:
            negative_score += 1
    positive_scores.append(positive_score)
    negative_scores.append(-1 * negative_score)

    # calculate polarity and subjectivity scores using TextBlob
    blob = TextBlob(text)
    polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
    subjectivity_score = (positive_score + negative_score) / ((len(words) - len(stop_words)) + 0.000001)
    polarity_scores.append(polarity_score)
    subjectivity_scores.append(subjectivity_score)

    # calculate fog index and average sentence length
    sentences = nltk.sent_tokenize(text)
    total_words = len(words)
    total_sentences = len(sentences)
    complex_words = []
    for word in words:
        syllable_count = 0  # initialize syllable_count with a default value
        if len(word) > 2 and word not in stop_words and re.match("^[a-zA-Z0-9_]*$", word): syllable_count = estimate(word)
        if syllable_count > 2:
            complex_words.append(word)

    total_complex_words = len(complex_words)
    average_sentence_length = total_words / total_sentences
    percentage_complex_words = (total_complex_words / total_words) * 100
    fog_index = 0.4 * (average_sentence_length + percentage_complex_words)
    fog_indexes.append(fog_index)
    average_sentence_lengths.append(average_sentence_length)
    percentage_complex_word.append(percentage_complex_words)

    # create a dictionary to store the results for each URL
results = {"URL_ID": [], "URL": [], "Positive Score": [], "Negative Score": [], "Polarity Score": [], "Subjectivity Score": [], "Average Number of Words Per Sentence": [], "Complex Word Count": [], "Word Count": [], "Syllable Count Per Word": [], "Personal Pronouns": [], "Average Word Length": []}

# iterate over each row in the DataFrame
for index, row in df.iterrows():
    url = row["URL"]
    
    # fetch the webpage content using urllib
    try:
        html = urllib.request.urlopen(url).read()
    except:
        print(f"Invalid URL: {url}")
        continue

    # parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html, "html.parser")

    # extract the article text
    text = ""
    for article in soup.find_all("article"):
        for p in article.find_all("p"):
            text += p.get_text()

    # clean the text by removing stop words and punctuation
    words = [word.lower() for word in nltk.word_tokenize(text) if word.isalpha()]
    words = [word for word in words if word not in stop_words]

    # calculate the various scores and metrics
    pscore = sum(1 for word in words if word in positive_words)
    nscore = sum(1 for word in words if word in negative_words) * -1
    ppscore = (pscore - nscore) / ((pscore + nscore) + 0.000001)
    subjscore = (pscore + nscore) / (len(words) + 0.000001)
    sent_count = len(nltk.sent_tokenize(text))
    word_count = len(words)
    avg_words_per_sent = word_count / sent_count
    
    complex_count = sum(1 for word in words if syllables(word) > 2)
    syllable_count = sum(syllables(word) for word in words)
    personal_pronouns = len(re.findall(r"\b(i|we|my|ours|us)\b", text, re.IGNORECASE))
    avg_word_length = sum(len(word) for word in words) / (len(words) + 0.000001)

    # append the results to the dictionary
    results["URL_ID"].append(row["URL_ID"])
    results["URL"].append(url)
    results["Positive Score"].append(pscore)
    results["Negative Score"].append(nscore)
    results["Polarity Score"].append(ppscore)
    results["Subjectivity Score"].append(subjscore)
    results["Average Number of Words Per Sentence"].append(avg_words_per_sent)
    results["Complex Word Count"].append(complex_count)
    results["Word Count"].append(word_count)
    results["Syllable Count Per Word"].append(syllable_count)
    results["Personal Pronouns"].append(personal_pronouns)
    results["Average Word Length"].append(avg_word_length)

# create a DataFrame from the results dictionary
results_df = pd.DataFrame(results)

# save the results to a Excel file
results_df.to_excel("Output10.xlsx", index=False)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Invalid URL: https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/
Invalid URL: https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/
Invalid URL: https://insights.blackcoffer.com/ensuring-growth-through-insurance-technology/
Invalid URL: https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/
Invalid URL: https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/
Invalid URL: https://insights.blackcoffer.com/ensuring-growth-through-insurance-technology/
